In [66]:
import numpy as np
import math as math
from numpy.typing import NDArray


In [59]:
class nueron_layer2:
    def __init__(self):
        self.act_out = 0
        self.lin_out = 0
        self.weights = np.random.rand(9)
    
    def activation(self): # Implement Hard Sigmoid
        return np.clip((self.lin_out + 1) / 2, 0, 1)
    
    def forward_pass(self, input: np.array):#size 8 
        self.lin_out = np.dot(input, self.weights)
        self.act_out = self.activation()
        

In [60]:
class nueron_layer1:
    def __init__(self):
        self.act_out = 0
        self.lin_out = 0
        self.weights = np.random.rand(10)
    
    def activation(self): # Implement ReLu
        return np.clip(self.lin_out, 0, None)
    
    def forward_pass(self, input: np.array):#size 1
        self.lin_out = np.dot(input, self.weights)
        self.act_out = self.activation()
        

In [61]:
def der_hard_sigmoid(x):
    if x > 1 or x < -1:
        return 0
    elif x > -1 and x < 1:
        return 1/2

def der_ReLu(x):

    if x > 0 :
        return 1
    elif x < 0:
        return 0


In [ ]:
class nn:

    def __init__(self):
        self.n2a = nueron_layer2()
        self.n1 = [nueron_layer1() for i in range(8)]

    def initalise_trainingdata(self, input:NDArray, labels, lr, niter):
        self.n = len(input)
        self.input = input
        self.labels = labels
        self.learning_rate = lr
        self.niter = niter # number of iterations in one epoch

    # input length 8
    def ForwardPassNueralNet(self, index=0):
        
        assert len(self.input[index]) == len(self.n1)
        
        for i in range(len(self.input[index])):      
            self.n1[i].forward_pass(np.array([self.input[index][i]]))
        self.n2a.forward_pass(np.array([j.act_out for j in self.n1]))


    def gradient_calc(self): # one epoch
    
    # Square Loss ~ (output[i] - label[i])**2 ~ Is The Loss Function 

        output = []
        gradient_vector = np.zeros(10*8 + 9) # first 8 are parameters related to the only nueron in nueron layer 2
                                    # last 8 are parameters related to the 8 nueron in nueron layer 1
        for i in range(len(self.input)):
            self.ForwardPassNueralNet(i)
            output.append(self.n2a.act_out)
            for param in range(9):
                gradient_vector[param] += 2*(self.n2a.act_out - self.labels[i])*der_hard_sigmoid(self.n2a.lin_out)*self.n1[param].act_out
            for param in range(8, 16):
                gradient_vector[param] += 2*(self.n2a.act_out - self.labels[i])*der_hard_sigmoid(self.n2a.lin_out)*self.n2a.weights[param-8]*der_ReLu(self.n1[param-8].lin_out)*self.input[i][param - 8]
            for i in range(len(self.n1)):
                for param in range(10):
        return gradient_vector
    

    def gradient_descent(self):


        for _ in range(self.niter):
            gradients = self.gradient_calc()
            for i in range(len(self.n1)):
                self.n1[i].weights -=  self.learning_rate*gradients[i+8]

            for i in range(len(self.n2a.weights)):
                self.n2a.weights -= self.learning_rate*gradients[i]

    
    def calc_loss(self):
        output = []
        # first 8 are parameters related to the only nueron in nueron layer 2
        # last 8 are parameters related to the 8 nueron in nueron layer 1
        
        for i in range(len(self.input)):
            self.ForwardPassNueralNet(i)
            output.append(self.n2a.act_out)

        return sum([(self.labels[i] - output[i])**2 for i in range(len(output))])


In [62]:
n2a = nueron_layer2()
# be careful with the input
n1 = [nueron_layer1() for i in range(8)]
n = 1000 # number of datapoints in training set

In [63]:
# input length 8
def ForwardPassNueralNet(input: np.array, n1 :list[nueron_layer1], n2a: nueron_layer2):
    
    assert len(input) == len(n1)
    
    for i in range(len(input)):      
        n1[i].forward_pass(np.array([input[i]]))

    n2a.forward_pass(np.array([j.act_out for j in n1]))


In [64]:
ForwardPassNueralNet([1,2,3,4,5,6,7,8], n1, n2a)
print(n2a.act_out)

1.0


In [68]:
def gradient_calc(input: NDArray, label, n1:list[nueron_layer1], n2a:nueron_layer2): # one epoch
    
    # Square Loss ~ (output[i] - label[i])**2 ~ Is The Loss Function 

    output = []
    gradient_vector = np.zeros(16) # first 8 are parameters related to the only nueron in nueron layer 2
                                   # last 8 are parameters related to the 8 nueron in nueron layer 1
    for i in range(len(input)):
        ForwardPassNueralNet(input[i], n1, n2a)
        output.append(n2a.act_out)
        for param in range(8):
            gradient_vector[param] += 2*(n2a.act_out - label[i])*der_hard_sigmoid(n2a.lin_out)*n1[param].act_out
        for param in range(8, 16):
            gradient_vector[param] += 2*(n2a.act_out - label[i])*der_hard_sigmoid(n2a.lin_out)*n2a.weights[param -8]*der_ReLu(n1[param-8].lin_out)*input[i][param - 8]
    
    return gradient_vector
    

In [ ]:
def gradient_descent(input: NDArray, label, n1:list[nueron_layer1], n2a:nueron_layer2):

    learning_rate = 0.1
    number_of_iters = 1000

    for _ in range(number_of_iters):
        gradients = gradient_calc(input, label, n1, n2a)
        for i in range(len(n1)):
            n1[i].weights -=  learning_rate*gradients[i+8]

        for i in range(len(n2a.weights)):
            n2a.weights -= learning_rate*gradients[i]

In [ ]:
def calc_loss(input: NDArray, label, n1:list[nueron_layer1], n2a:nueron_layer2):
    output = []
    # first 8 are parameters related to the only nueron in nueron layer 2
    # last 8 are parameters related to the 8 nueron in nueron layer 1
    
    for i in range(len(input)):
        ForwardPassNueralNet(input[i], n1, n2a)
        output.append(n2a.act_out)

    return sum([(label[i] - output[i])**2 for i in range(len(output))])